In [1]:
! pip install -q datasets
! pip install -q pandas
! pip install -q wikipedia

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset
import pandas as pd
from datasets import Dataset, DatasetDict
import numpy as np

In [4]:
dataset = load_dataset(
    "lmqg/qa_harvesting_from_wikipedia", split="train+test+validation"
)

In [8]:
df = pd.DataFrame(dataset)

In [7]:
print(df["title"].nunique())

8912


In [34]:
article_titles = df["title"].unique()

In [5]:
import wikipedia

article_cache = {}


def get_full_article(article):
    global article_cache

    if article not in article_cache:
        results = wikipedia.search(article)
        title = results[0]
        page = wikipedia.page(title, auto_suggest=False)
        article_cache[article] = page.content


def attach_article(row):
    global article_cache

    if row["title"] in article_cache:
        row["article"] = article_cache[row["title"]]

    return row

In [36]:
from wikipedia import DisambiguationError

failed_articles = []

for id, article in enumerate(article_titles):
    try:
        get_full_article(article)
        if id % 500 == 0:
            print(f"Processed {id} articles")
    except DisambiguationError:
        failed_articles.append(article)

Processed 0 articles
Processed 500 articles
Processed 1000 articles


/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Processed 1500 articles
Processed 2000 articles
Processed 2500 articles
Processed 3000 articles
Processed 3500 articles
Processed 4000 articles
Processed 4500 articles
Processed 5000 articles
Processed 5500 articles
Processed 6000 articles
Processed 6500 articles
Processed 7000 articles
Processed 7500 articles
Processed 8000 articles
Processed 8500 articles


In [37]:
print(len(failed_articles))
print(failed_articles)

24
['LIBRIS', 'Alcohol', 'Poles', 'Asphalt', 'Industry', 'First Nations', 'SIMBAD', 'Solution', 'Unorganized territory', 'Saba', "Queen's University", 'Diving', 'National League', 'Admiralty', 'Presenter', 'Media Source', 'Viacom', 'Retraction', 'Gemeinde', 'Dualism', 'Tripoli', 'Barony', 'Licentiate', 'Prior']


In [ ]:
import pandas as pd
from datasets import Dataset

# convert dictionary to dataframe
df = pd.DataFrame(list(article_cache.items()), columns=["title", "article"])

# convert dataframe to dataset
article_dataset = Dataset.from_pandas(df)
article_dataset.push_to_hub("legacy107/articles_for_qa_wikipedia")

In [13]:
article_dataset = load_dataset("legacy107/articles_for_qa_wikipedia", split="train")

article_df = article_dataset.to_pandas()

# convert dataframe to dictionary
article_cache = {x[0]: x[1] for x in article_df.itertuples(index=False)}

In [7]:
dataset = dataset.filter(
    lambda row: row["title"] in article_cache.keys()
    and row["context"] in article_cache[row["title"]]
)
dataset

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 173344
})

In [9]:
offset = 10
split_ratio = 0.1
train_size = df.shape[0] * (1 - split_ratio * 2)
test_size = df.shape[0] * split_ratio
train_df = []
test_df = []
val_df = []
unique_contexts = df["title"].unique()
np.random.shuffle(unique_contexts)
prev_idx = 0
idx = 1
while True:
    train_df = df[df["title"].isin(unique_contexts[prev_idx:idx])]
    idx += 1

    if train_size - offset < train_df.shape[0]:
        prev_idx = idx - 1
        break

while True:
    test_df = df[df["title"].isin(unique_contexts[prev_idx:idx])]
    idx += 1

    if test_size - offset < test_df.shape[0]:
        prev_idx = idx - 1
        break

val_df = df[df["title"].isin(unique_contexts[prev_idx:])]
print(len(train_df), len(test_df), len(val_df))

138712 17341 17291


In [10]:
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)
val_ds = Dataset.from_pandas(val_df)

In [11]:
dataset = DatasetDict({"train": train_ds, "test": test_ds, "validation": val_ds})

In [44]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 138678
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 17373
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 17293
    })
})

In [ ]:
dataset.push_to_hub("legacy107/qa_wikipedia")

In [ ]:
dataset = dataset.map(attach_article, batched=False)

In [14]:
dataset = dataset.flatten()
dataset = dataset.rename_column("answers.answer_start", "answer_start")
dataset = dataset.rename_column("answers.text", "answer")

In [ ]:
def flatten(row):
    row["answer_start"] = row["answer_start"][0]
    row["answer"] = row["answer"][0]
    return row


dataset = dataset.map(flatten, batched=False)

In [ ]:
dataset["train"][0]

In [ ]:
dataset.push_to_hub("legacy107/qa_wikipedia")

In [27]:
dataset = dataset.remove_columns("article")

In [ ]:
dataset.push_to_hub("legacy107/qa_wikipedia_no_article")